# MATRIX

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

import math

class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903
    """
    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat

        self.W = nn.Parameter(torch.empty(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.empty(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, h, adj):
        Wh = torch.mm(h, self.W) # h.shape: (N, in_features), Wh.shape: (N, out_features)
        e = self._prepare_attentional_mechanism_input(Wh)

        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime = torch.matmul(attention, Wh)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

    def _prepare_attentional_mechanism_input(self, Wh):
        # Wh.shape (N, out_feature)
        # self.a.shape (2 * out_feature, 1)
        # Wh1&2.shape (N, 1)
        # e.shape (N, N)
        Wh1 = torch.matmul(Wh, self.a[:self.out_features, :])
        Wh2 = torch.matmul(Wh, self.a[self.out_features:, :])
        # broadcast add
        e = Wh1 + Wh2.T
        return self.leakyrelu(e)

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'


In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
import os

def normalize_features(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

def accuracy(output, labels):
    probs = torch.exp(output)
    preds = torch.argmax(probs, dim = 1)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def load_multi_data(folder, att_file, edge_list_name):
    att = pd.read_csv(folder + att_file)
    edge_list = []
    for name in edge_list_name:
        edge_list.append(pd.read_csv(folder + name))

    #get y and x
    labels = np.array(att["ever_pos"])
    features = sp.csr_matrix(att[["# Insert feature variables here"]])    
    #features = normalize_features(features)

    #get adj mat
    adj_list = []
    for edge in edge_list:
        #get row col idx for adj matrix
        row_idx = []
        col_idx = []
        for i in range(edge.shape[0]):
            id_from = edge.iloc[i,0]
            id_to = edge.iloc[i,1]
            # uid as unique identifier
            row_id = att.index[att["uid"] == id_from]
            row_idx.append(row_id[0])
            col_id = att.index[att["uid"] == id_to]
            col_idx.append(col_id[0])

        if edge.shape[1] == 2:
            adj = sp.coo_matrix((np.ones(edge.shape[0]), (row_idx, col_idx)), shape=(att.shape[0], att.shape[0]), dtype=np.float32)

        elif edge.shape[1] == 3:
            adj = sp.coo_matrix((np.array(edge.iloc[:,2]), (row_idx, col_idx)), shape=(att.shape[0], att.shape[0]), dtype=np.float32)

        #make adj symmetric
        adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
        #normaliza adj
        adj = normalize_adj(adj + sp.eye(adj.shape[0]))
        adj = torch.FloatTensor(np.array(adj.todense()))
        adj_list.append(adj)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(labels)

    dim = len(labels)
    idx_train = range(1585)
    idx_test = range(1585, dim)

    return adj_list, features, labels, idx_train, idx_test

def load_data(folder, att_file, edge_name):
    att = pd.read_csv(folder + att_file)
    edge_list = pd.read_csv(folder + edge_name)

    #get y and x
    labels = np.array(att["y"])
    # b,c,d are age groups
    features = sp.csr_matrix(att[["# Insert feature variables here"]])
    #features = normalize_features(features)

    #get adj mat
    row_idx = []
    col_idx = []
    for i in range(edge_list.shape[0]):
        id_from = edge_list.iloc[i,0]
        id_to = edge_list.iloc[i,1]
        row_id = att.index[att["uid"] == id_from]
        row_idx.append(row_id[0])
        col_id = att.index[att["uid"] == id_to]
        col_idx.append(col_id[0])

    if edge_list.shape[1] == 2:
        adj = sp.coo_matrix((np.ones(edge_list.shape[0]), (row_idx, col_idx)), shape=(att.shape[0], att.shape[0]), dtype=np.float32)


        #make adj symmetric
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
        #normaliza adj
    adj = normalize_adj(adj + sp.eye(adj.shape[0]))
    adj = torch.FloatTensor(np.array(adj.todense()))

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(labels)

    dim = len(labels)
    idx_train = range(1585)
    idx_test = range(1585, dim)

    return adj, features, labels, idx_train, idx_test

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F



class GAT(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout, alpha, nheads):
        """Dense version of GAT."""
        super(GAT, self).__init__()
        self.dropout = dropout

        self.attentions = [GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)

        self.out_att = nn.Linear(nhid * nheads, nclass)

    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training=self.training)
        x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.elu(self.out_att(x))
        return F.log_softmax(x, dim=1)



class FusionGAT(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout, alpha, adj_list, nheads):
        super(FusionGAT, self).__init__()

        self.dropout = dropout
        self.nheads = nheads
        self.adj_list = adj_list

        # Define list of GAT layers for each adjacency matrix
        self.attentions = nn.ModuleList()
        for i in range(len(adj_list)):
            att_list = nn.ModuleList([GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)])
            self.attentions.append(att_list)
            for k, attention in enumerate(self.attentions):
                self.add_module('adj{}, attention_{}'.format(i, k), attention)

        # Define linear layer for integration with L1 regularization
        self.integration_att = nn.Linear(nhid * nheads, nclass)
        self.fusion = nn.Linear(nclass * len(adj_list), nclass)
        self.l1_reg = nn.L1Loss(reduction='mean')

    def forward(self, x, adj_list):
        x = F.dropout(x, self.dropout, training=self.training)
        # Compute output for each adjacency matrix using GAT layers
        output_list = []
        for i, adj in enumerate(adj_list):
            x_i = torch.cat([att(x, adj) for att in self.attentions[i]], dim=1)
            x_i = F.dropout(x_i, self.dropout, training=self.training)
            x_i = F.elu(self.integration_att(x_i))
            output_list.append(x_i)
        output = torch.cat(output_list, dim=1)

        # Apply linear layer for integration with L1 regularization
        output = F.dropout(output, self.dropout, training=self.training)
        output = self.fusion(output.view(output.size(0), -1))
        l1_loss = self.l1_reg(self.fusion.weight, torch.zeros_like(self.fusion.weight))
        return F.log_softmax(output, dim=1), l1_loss



class FusionGAT2(nn.Module):
    def __init__(self, nfeat, nhid1, nhid2, nclass, dropout, alpha, adj_list, nheads):
        super(FusionGAT2, self).__init__()

        self.dropout = dropout
        self.nheads = nheads
        self.adj_list = adj_list

        # Define list of GAT layers for each adjacency matrix in attention 1
        self.attentions1 = nn.ModuleList()
        for i in range(len(adj_list)):
            att_list = nn.ModuleList([GraphAttentionLayer(nfeat, nhid1, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)])
            self.attentions1.append(att_list)
            for k, attention in enumerate(self.attentions1):
                self.add_module('adj{}, attention_layer1_{}'.format(i, k), attention)

        # Define linear layer for integration of multihead attention1
        self.integration_att1 = nn.Linear(nhid1 * nheads, nhid1)

        self.attentions2 = nn.ModuleList()
        for i in range(len(adj_list)):
            att_list2 = nn.ModuleList([GraphAttentionLayer(nhid1, nhid2, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)])
            self.attentions2.append(att_list2)
            for k, attention in enumerate(self.attentions2):
                self.add_module('adj{}, attention_layer2_{}'.format(i, k), attention)

        # Define linear layer for integration of multihead attention2
        self.integration_att2 = nn.Linear(nhid2 * nheads, nclass)

        #fusion layer with l1 penalty
        self.fusion_att = nn.Linear(nclass * len(adj_list), nclass)
        self.l1_reg = nn.L1Loss(reduction='mean')

    def forward(self, x, adj_list):
        x = F.dropout(x, self.dropout, training=self.training)
        # Compute output for each adjacency matrix using GAT layers
        output_list = []
        for i, adj in enumerate(adj_list):
            #attention layer 1
            x_i = torch.cat([att(x, adj) for att in self.attentions1[i]], dim=1)
            x_i = F.dropout(x_i, self.dropout, training=self.training)
            x_i = F.elu(self.integration_att1(x_i))

            #attention layer 2
            x_i = torch.cat([att(x_i, adj) for att in self.attentions2[i]], dim=1)
            x_i = F.dropout(x_i, self.dropout, training=self.training)
            x_i = F.elu(self.integration_att2(x_i))
            output_list.append(x_i)

        output = torch.cat(output_list, dim=1)
        output = F.dropout(output, self.dropout, training=self.training)
        output = self.fusion_att(output.view(output.size(0), -1))
        l1_loss = self.l1_reg(self.fusion_att.weight, torch.zeros_like(self.fusion_att.weight))
        return F.log_softmax(output, dim=1), l1_loss



class FusionGAT3(nn.Module):
    def __init__(self, nfeat, nhid1, nhid2, fusion1_dim, nclass, dropout, alpha, adj_list, nheads):
        super(FusionGAT3, self).__init__()

        self.dropout = dropout
        self.nheads = nheads
        self.adj_list = adj_list

        # Define list of GAT layers for each adjacency matrix
        #att 1
        self.attentions1 = nn.ModuleList()
        for i in range(len(adj_list)):
            att_list = nn.ModuleList([GraphAttentionLayer(nfeat, nhid1, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)])
            self.attentions1.append(att_list)
            for k, attention in enumerate(self.attentions1):
                self.add_module('adj{}, attention_layer1_{}'.format(i, k), attention)

        # fusion1
        self.integration_att1 = nn.Linear(nhid1 * nheads, fusion1_dim)
        self.fusion_att1 = nn.Linear(fusion1_dim * len(adj_list), fusion1_dim)
        self.l1_reg1 = nn.L1Loss(reduction='mean')

        #att 2
        self.attentions2 = nn.ModuleList()
        for i in range(len(adj_list)):
            att_list = nn.ModuleList([GraphAttentionLayer(fusion1_dim, nhid2, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)])
            self.attentions2.append(att_list)
            for k, attention in enumerate(self.attentions2):
                self.add_module('adj{}, attention_layer2_{}'.format(i, k), attention)

        #fusion2
        self.integration_att2 = nn.Linear(nhid2 * nheads, nclass)
        self.fusion_att2 = nn.Linear(nclass * len(adj_list), nclass)
        self.l1_reg2 = nn.L1Loss(reduction='mean')


    def forward(self, x, adj_list):
        x = F.dropout(x, self.dropout, training=self.training)

        # Compute output for each adjacency matrix using GAT layers
        output_list = []
        for i, adj in enumerate(adj_list):
            x_i = x
            x_i = torch.cat([att(x, adj) for att in self.attentions1[i]], dim=1)
            x_i = F.dropout(x_i, self.dropout, training=self.training)
            x_i = F.elu(self.integration_att1(x_i))
            output_list.append(x_i)
        output = torch.cat(output_list, dim=1)

        # Apply linear layer for integration with L1 regularization
        output = F.dropout(output, self.dropout, training=self.training)
        output = self.fusion_att1(output.view(output.size(0), -1))
        l1_loss1 = self.l1_reg1(self.fusion_att1.weight, torch.zeros_like(self.fusion_att1.weight))


        output_list2 = []
        for i, adj in enumerate(adj_list):
            x_i = torch.cat([att(output, adj) for att in self.attentions2[i]], dim=1)
            x_i = F.dropout(x_i, self.dropout, training=self.training)
            x_i = F.elu(self.integration_att2(x_i))
            output_list2.append(x_i)
        output2 = torch.cat(output_list2, dim=1)

        # Apply linear layer for integration with L1 regularization
        output2 = F.dropout(output2, self.dropout, training=self.training)
        output2 = self.fusion_att2(output2.view(output.size(0), -1))
        l1_loss2 = self.l1_reg2(self.fusion_att2.weight, torch.zeros_like(self.fusion_att2.weight))

        return F.log_softmax(output2, dim=1), l1_loss1 + l1_loss2

class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

In [ ]:
import os
import glob
import time
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import precision_recall_curve
import time
import json
from sklearn import metrics
#from load import accuracy, load_multi_data
#from models import FusionGAT3


seed = 72
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output, l1_loss = model(features, adj_list_tt)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train]) + l1_loss*lambda_l1
    acc_train = accuracy(output[idx_train], labels[idx_train])
    loss_train.backward()
    optimizer.step()

    loss_test = F.nll_loss(output[idx_test], labels[idx_test]) + l1_loss*lambda_l1
    acc_test = accuracy(output[idx_test], labels[idx_test])
    y_prob = torch.exp(output[:, 1]).cpu().detach().numpy()

    auc_score = roc_auc_score(labels[idx_test].cpu().detach().numpy(), y_prob[idx_test.cpu().detach().numpy()])
    print('Epoch: {:03d}'.format(epoch+1),
          'loss_train: {:.3f}'.format(loss_train.data.item()),
          'acc_train: {:.3f}'.format(acc_train.data.item()),
          'loss_test: {:.3f}'.format(loss_test.data.item()),
          'acc_test: {:.3f}'.format(acc_test.data.item()),
          "AUC = {:.2f}".format(auc_score),
          'time: {:.3f}s'.format(time.time() - t))

    return loss_test.data.item(), acc_test, auc_score

def compute_test(model):
    model.eval()
    output, l1_loss = model(features, adj_list_tt)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test]) + l1_loss*lambda_l1
    acc_test = accuracy(output[idx_test], labels[idx_test])
    y_prob = torch.exp(output[:, 1]).cpu().detach().numpy()

    auc_score = roc_auc_score(labels[idx_test].cpu().detach().numpy(), y_prob[idx_test.cpu().detach().numpy()])
    # compute false alarm rate and f1 score
    y_pred = np.array([0 if p < 0.5 else 1 for p in y_prob])
    false_alarm_rate = np.mean(y_pred[idx_test.cpu().detach().numpy()] == 1)
    f1score = f1_score(labels[idx_test].cpu().detach().numpy(), y_pred[idx_test.cpu().detach().numpy()])
    precisionn, recalll, _ = precision_recall_curve(labels[idx_test].cpu().detach().numpy(), y_prob[idx_test.cpu().detach().numpy()])

    pr_auc = metrics.auc(recalll, precisionn)
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.data.item()),
          "accuracy= {:.4f}".format(acc_test.data.item()),
          "AUC = {:.2f}".format(auc_score),
          "False Alarm Rate = {:.4f}".format(false_alarm_rate),
          "F1 Score = {:.4f}".format(f1score))
    return loss_test.data.item(), acc_test.data.item(), auc_score, false_alarm_rate, f1score, y_pred, pr_auc


#parameter setting
#

cuda = True
epochs = 500
#lrs = [0.0005, 0.001]
#weight_decays = [5e-5, 5e-4]
#dropouts = [0.3, 0.5]
#hidden_dims1 = [64]
#hidden_dims2 = [64]
#fusion1_dims = [4]
#nb_heads = [8, 12]
#alpha = 0.2 #Alpha for the leaky_relu
#lambda_l1s = [0.0001, 0.001, 0.01]
#patience = 50

lrs = [0.0005]
weight_decays = [0.00005]
dropouts = [0.3]
hidden_dims1 = [16]
hidden_dims2 = [16]
fusion1_dims = [4]
nb_heads = [8]
alpha = 0.2 #Alpha for the leaky_relu
lambda_l1s = [0.0001, 0.001, 0.01]
patience = 50


# Load data
sv_path = ""
folder = "# Insert data folder pathway here"
att_name = "# Insert main interaction attribute dataset name here"
edge_list_name = ["# Insert all network edge list dataset names here"]

adj_list, features, labels, idx_train, idx_test = load_multi_data(folder, att_name, edge_list_name)

adj_list_tensor = torch.stack(adj_list)
features, adj_list_tt, labels = Variable(features), adj_list_tensor, Variable(labels)

count = 0
for lr in lrs:
    for dropout in dropouts:
        for hid1 in hidden_dims1:
            for hid2 in hidden_dims2:
                for fusion1_dim in fusion1_dims:
                    for nb_head in nb_heads:
                        for weight_decay in weight_decays:
                            for lambda_l1 in lambda_l1s:

                                # Model and optimizer
                                model = FusionGAT3(nfeat=features.shape[1],
                                            nhid1=hid1,
                                            nhid2=hid2,
                                            fusion1_dim = fusion1_dim,
                                            nclass=int(labels.max()) + 1,
                                            dropout=dropout,
                                            alpha=alpha,
                                            adj_list= adj_list_tt,
                                            nheads = nb_head)

                                optimizer = optim.Adam(model.parameters(),
                                                    lr=lr,
                                                    weight_decay=weight_decay)

                                if cuda:
                                    model.cuda()
                                    features = features.cuda()
                                    adj_list_tt = adj_list_tt.cuda()
                                    labels = labels.cuda()
                                    idx_train = torch.tensor(idx_train).cuda()
                                    idx_test = torch.tensor(idx_test).cuda()


                                # Train model
                                t_total = time.time()
                                bad_counter = 0
                                best_auc = -1
                                best_epoch = 0
                                for epoch in range(epochs):
                                    loss, acc, auc = train(epoch)

                                    torch.save(model.state_dict(), sv_path + '{}.pkl'.format(epoch))
                                    if auc > best_auc:
                                        best_auc = auc
                                        best_epoch = epoch
                                        bad_counter = 0
                                    else:
                                        bad_counter += 1

                                    if bad_counter == patience:
                                        break

                                files = glob.glob(sv_path +'*.pkl')
                                for file in files:
                                    filename = file.split('/')[-1]
                                    epoch_nb = int(filename.split('.')[0])
                                    if epoch_nb != best_epoch:
                                        os.remove(file)

                                print("Optimization Finished!")
                                print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

                                # Restore best model
                                print('Loading {}th epoch'.format(best_epoch))
                                model.load_state_dict(torch.load(sv_path +'{}.pkl'.format(best_epoch)))

                                # Testing
                                test_loss, test_acc, auc, far, f1, y_pred, pr_auc= compute_test(model)
                                hyper_para = {}
                                hyper_para["lr"] = lr
                                hyper_para["weight_decay"] = weight_decay
                                hyper_para["dropout"] = dropout
                                hyper_para["hidden_dim1"] = hid1
                                hyper_para["hidden_dim2"] = hid2
                                hyper_para["lambda"] = lambda_l1
                                hyper_para["fusion1_dim"] = fusion1_dim
                                hyper_para["nb_heads"] = nb_head
                                hyper_para["alpha"] = alpha
                                hyper_para["loss"] = test_loss
                                hyper_para["accuracy"] = test_acc
                                hyper_para["auc"] = auc
                                hyper_para["pr_auc"] = pr_auc
                                hyper_para["false_alarm_rate"] = far
                                hyper_para["f1_score"] = f1
                                with open(sv_path +"hyperpara.json", "a+") as fp:
                                    fp.write('\n')
                                    json.dump(hyper_para, fp)

Epoch: 001 loss_train: 0.693 acc_train: 0.536 loss_test: 0.702 acc_test: 0.517 AUC = 0.52 time: 2.365s
Epoch: 002 loss_train: 0.693 acc_train: 0.560 loss_test: 0.699 acc_test: 0.517 AUC = 0.50 time: 0.493s
Epoch: 003 loss_train: 0.688 acc_train: 0.560 loss_test: 0.707 acc_test: 0.521 AUC = 0.48 time: 0.494s
Epoch: 004 loss_train: 0.684 acc_train: 0.582 loss_test: 0.712 acc_test: 0.474 AUC = 0.47 time: 0.494s
Epoch: 005 loss_train: 0.684 acc_train: 0.565 loss_test: 0.704 acc_test: 0.532 AUC = 0.52 time: 0.493s
Epoch: 006 loss_train: 0.675 acc_train: 0.602 loss_test: 0.704 acc_test: 0.545 AUC = 0.53 time: 0.493s
Epoch: 007 loss_train: 0.681 acc_train: 0.604 loss_test: 0.690 acc_test: 0.535 AUC = 0.55 time: 0.494s
Epoch: 008 loss_train: 0.672 acc_train: 0.631 loss_test: 0.704 acc_test: 0.560 AUC = 0.53 time: 0.494s
Epoch: 009 loss_train: 0.665 acc_train: 0.632 loss_test: 0.699 acc_test: 0.543 AUC = 0.54 time: 0.494s
Epoch: 010 loss_train: 0.664 acc_train: 0.650 loss_test: 0.697 acc_test: 

<ipython-input-5-a4368b6b2731>:179: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(sv_path +'{}.pkl'.format(best_epoch)))


Test set results: loss= 0.4232 accuracy= 0.7835 AUC = 0.90 False Alarm Rate = 0.3078 F1 Score = 0.7189


<ipython-input-5-a4368b6b2731>:144: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idx_train = torch.tensor(idx_train).cuda()
<ipython-input-5-a4368b6b2731>:145: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idx_test = torch.tensor(idx_test).cuda()


Epoch: 001 loss_train: 0.797 acc_train: 0.303 loss_test: 0.738 acc_test: 0.461 AUC = 0.45 time: 0.498s
Epoch: 002 loss_train: 0.787 acc_train: 0.313 loss_test: 0.730 acc_test: 0.461 AUC = 0.47 time: 0.498s
Epoch: 003 loss_train: 0.786 acc_train: 0.299 loss_test: 0.739 acc_test: 0.457 AUC = 0.44 time: 0.499s
Epoch: 004 loss_train: 0.766 acc_train: 0.329 loss_test: 0.720 acc_test: 0.474 AUC = 0.51 time: 0.500s
Epoch: 005 loss_train: 0.760 acc_train: 0.336 loss_test: 0.734 acc_test: 0.451 AUC = 0.43 time: 0.498s
Epoch: 006 loss_train: 0.755 acc_train: 0.338 loss_test: 0.727 acc_test: 0.458 AUC = 0.44 time: 0.499s
Epoch: 007 loss_train: 0.751 acc_train: 0.332 loss_test: 0.719 acc_test: 0.462 AUC = 0.47 time: 0.498s
Epoch: 008 loss_train: 0.747 acc_train: 0.362 loss_test: 0.729 acc_test: 0.434 AUC = 0.46 time: 0.498s
Epoch: 009 loss_train: 0.741 acc_train: 0.363 loss_test: 0.710 acc_test: 0.471 AUC = 0.51 time: 0.498s
Epoch: 010 loss_train: 0.739 acc_train: 0.396 loss_test: 0.716 acc_test: 

...

Test set results: loss= 0.5819 accuracy= 0.6878 AUC = 0.80 False Alarm Rate = 0.2975 F1 Score = 0.5891


<ipython-input-5-a4368b6b2731>:144: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idx_train = torch.tensor(idx_train).cuda()
<ipython-input-5-a4368b6b2731>:145: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  idx_test = torch.tensor(idx_test).cuda()


Epoch: 001 loss_train: 0.785 acc_train: 0.311 loss_test: 0.733 acc_test: 0.457 AUC = 0.47 time: 0.498s
Epoch: 002 loss_train: 0.781 acc_train: 0.318 loss_test: 0.724 acc_test: 0.468 AUC = 0.50 time: 0.498s
Epoch: 003 loss_train: 0.767 acc_train: 0.326 loss_test: 0.724 acc_test: 0.486 AUC = 0.50 time: 0.501s
Epoch: 004 loss_train: 0.769 acc_train: 0.319 loss_test: 0.733 acc_test: 0.452 AUC = 0.46 time: 0.498s
Epoch: 005 loss_train: 0.765 acc_train: 0.315 loss_test: 0.733 acc_test: 0.442 AUC = 0.47 time: 0.498s
Epoch: 006 loss_train: 0.764 acc_train: 0.323 loss_test: 0.717 acc_test: 0.470 AUC = 0.49 time: 0.498s
Epoch: 007 loss_train: 0.758 acc_train: 0.322 loss_test: 0.715 acc_test: 0.490 AUC = 0.51 time: 0.499s
Epoch: 008 loss_train: 0.754 acc_train: 0.322 loss_test: 0.716 acc_test: 0.467 AUC = 0.49 time: 0.498s
Epoch: 009 loss_train: 0.749 acc_train: 0.336 loss_test: 0.719 acc_test: 0.446 AUC = 0.48 time: 0.498s
Epoch: 010 loss_train: 0.744 acc_train: 0.344 loss_test: 0.717 acc_test: 

<ipython-input-5-a4368b6b2731>:179: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(sv_path +'{}.pkl'.format(best_epoch)))


Test set results: loss= 0.4425 accuracy= 0.7688 AUC = 0.89 False Alarm Rate = 0.3019 F1 Score = 0.6975


# Explainer

In [ ]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
import torch
import os
from torch.utils import data
from torch.autograd import Variable

def normalize_features(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

def normalize_adj(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv_sqrt = np.power(rowsum, -0.5).flatten()
    r_inv_sqrt[np.isinf(r_inv_sqrt)] = 0.
    r_mat_inv_sqrt = sp.diags(r_inv_sqrt)
    return mx.dot(r_mat_inv_sqrt).transpose().dot(r_mat_inv_sqrt)

def accuracy(output, labels):
    probs = torch.exp(output)
    preds = torch.argmax(probs, dim = 1)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)


def load_multi_data(folder, att_file, edge_list_name):
    print("loading data")
    att = pd.read_csv(folder + att_file)
    edge_list = []
    for name in edge_list_name:
        edge_list.append(pd.read_csv(folder + name))

    #get y and x
    labels = np.array(att["# Insert outcome variable here"])
    features = sp.csr_matrix(att[["# Insert feature variables here"]])
    #features = normalize_features(features)

    #get adj mat
    adj_list = []
    for edge in edge_list:
        #get row col idx for adj matrix
        row_idx = []
        col_idx = []
        for i in range(edge.shape[0]):
            id_from = edge.iloc[i,0]
            id_to = edge.iloc[i,1]
            row_id = att.index[att["uid"] == id_from]
            row_idx.append(row_id[0])
            col_id = att.index[att["uid"] == id_to]
            col_idx.append(col_id[0])

        if edge.shape[1] == 2:
            adj = sp.coo_matrix((np.ones(edge.shape[0]), (row_idx, col_idx)), shape=(att.shape[0], att.shape[0]), dtype=np.float32)

        elif edge.shape[1] == 3:
            adj = sp.coo_matrix((np.array(edge.iloc[:,2]), (row_idx, col_idx)), shape=(att.shape[0], att.shape[0]), dtype=np.float32)

        #make adj symmetric
        adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
        #normaliza adj
        adj = normalize_adj(adj + sp.eye(adj.shape[0]))
        adj = torch.FloatTensor(np.array(adj.todense()))
        adj_list.append(adj)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(labels)

    dim = len(labels)
    idx_train = range(1585)
    idx_test = range(1585, dim)

    features, adj_list, labels = Variable(features), torch.stack(adj_list), Variable(labels)


    data = MyDataset(adj_list, features, labels, idx_train, idx_test)
    return data

class MyDataset(data.Dataset):
    def __init__(self, adj_ls, node_features, labels, idx_train, idx_test):
        self.adj_ls = adj_ls
        self.features = node_features
        self.train_mask = idx_train
        self.test_mask = idx_test
        self.labels = labels

    def __getitem__(self, index):
        node_features = self.features[index]
        labels = self.labels[index]
        return self.adj_ls, node_features, labels

    def __len__(self):
        return len(self.features)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F



class GAT(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout, alpha, nheads):
        """Dense version of GAT."""
        super(GAT, self).__init__()
        self.dropout = dropout

        self.attentions = [GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)]
        for i, attention in enumerate(self.attentions):
            self.add_module('attention_{}'.format(i), attention)

        self.out_att = nn.Linear(nhid * nheads, nclass)

    def forward(self, x, adj):
        x = F.dropout(x, self.dropout, training=self.training)
        x = torch.cat([att(x, adj) for att in self.attentions], dim=1)
        x = F.dropout(x, self.dropout, training=self.training)
        x = F.elu(self.out_att(x))
        return F.log_softmax(x, dim=1)



class FusionGAT(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout, alpha, adj_list, nheads):
        super(FusionGAT, self).__init__()

        self.dropout = dropout
        self.nheads = nheads
        self.adj_list = adj_list

        # Define list of GAT layers for each adjacency matrix
        self.attentions = nn.ModuleList()
        for i in range(len(adj_list)):
            att_list = nn.ModuleList([GraphAttentionLayer(nfeat, nhid, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)])
            self.attentions.append(att_list)
            for k, attention in enumerate(self.attentions):
                self.add_module('adj{}, attention_{}'.format(i, k), attention)

        # Define linear layer for integration with L1 regularization
        self.integration_att = nn.Linear(nhid * nheads, nclass)
        self.fusion = nn.Linear(nclass * len(adj_list), nclass)
        self.l1_reg = nn.L1Loss(reduction='mean')

    def forward(self, x, adj_list):
        x = F.dropout(x, self.dropout, training=self.training)
        # Compute output for each adjacency matrix using GAT layers
        output_list = []
        for i, adj in enumerate(adj_list):
            x_i = torch.cat([att(x, adj) for att in self.attentions[i]], dim=1)
            x_i = F.dropout(x_i, self.dropout, training=self.training)
            x_i = F.elu(self.integration_att(x_i))
            output_list.append(x_i)
        output = torch.cat(output_list, dim=1)

        # Apply linear layer for integration with L1 regularization
        output = F.dropout(output, self.dropout, training=self.training)
        output = self.fusion(output.view(output.size(0), -1))
        l1_loss = self.l1_reg(self.fusion.weight, torch.zeros_like(self.fusion.weight))
        return F.log_softmax(output, dim=1), l1_loss



class FusionGAT2(nn.Module):
    def __init__(self, nfeat, nhid1, nhid2, nclass, dropout, alpha, adj_list, nheads):
        super(FusionGAT2, self).__init__()

        self.dropout = dropout
        self.nheads = nheads
        self.adj_list = adj_list

        # Define list of GAT layers for each adjacency matrix in attention 1
        self.attentions1 = nn.ModuleList()
        for i in range(len(adj_list)):
            att_list = nn.ModuleList([GraphAttentionLayer(nfeat, nhid1, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)])
            self.attentions1.append(att_list)
            for k, attention in enumerate(self.attentions1):
                self.add_module('adj{}, attention_layer1_{}'.format(i, k), attention)

        # Define linear layer for integration of multihead attention1
        self.integration_att1 = nn.Linear(nhid1 * nheads, nhid1)

        self.attentions2 = nn.ModuleList()
        for i in range(len(adj_list)):
            att_list2 = nn.ModuleList([GraphAttentionLayer(nhid1, nhid2, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)])
            self.attentions2.append(att_list2)
            for k, attention in enumerate(self.attentions2):
                self.add_module('adj{}, attention_layer2_{}'.format(i, k), attention)

        # Define linear layer for integration of multihead attention2
        self.integration_att2 = nn.Linear(nhid2 * nheads, nclass)

        #fusion layer with l1 penalty
        self.fusion_att = nn.Linear(nclass * len(adj_list), nclass)
        self.l1_reg = nn.L1Loss(reduction='mean')

    def forward(self, x, adj_list):
        x = F.dropout(x, self.dropout, training=self.training)
        # Compute output for each adjacency matrix using GAT layers
        output_list = []
        for i, adj in enumerate(adj_list):
            #attention layer 1
            x_i = torch.cat([att(x, adj) for att in self.attentions1[i]], dim=1)
            x_i = F.dropout(x_i, self.dropout, training=self.training)
            x_i = F.elu(self.integration_att1(x_i))

            #attention layer 2
            x_i = torch.cat([att(x_i, adj) for att in self.attentions2[i]], dim=1)
            x_i = F.dropout(x_i, self.dropout, training=self.training)
            x_i = F.elu(self.integration_att2(x_i))
            output_list.append(x_i)

        output = torch.cat(output_list, dim=1)
        output = F.dropout(output, self.dropout, training=self.training)
        output = self.fusion_att(output.view(output.size(0), -1))
        l1_loss = self.l1_reg(self.fusion_att.weight, torch.zeros_like(self.fusion_att.weight))
        return F.log_softmax(output, dim=1), l1_loss



class FusionGAT3(nn.Module):
    def __init__(self, nfeat, nhid1, nhid2, fusion1_dim, nclass, dropout, alpha, adj_list, nheads):
        super(FusionGAT3, self).__init__()

        self.dropout = dropout
        self.nheads = nheads
        self.adj_list = adj_list

        # Define list of GAT layers for each adjacency matrix
        #att 1
        self.attentions1 = nn.ModuleList()
        for i in range(len(adj_list)):
            att_list = nn.ModuleList([GraphAttentionLayer(nfeat, nhid1, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)])
            self.attentions1.append(att_list)
            for k, attention in enumerate(self.attentions1):
                self.add_module('adj{}, attention_layer1_{}'.format(i, k), attention)

        # fusion1
        self.integration_att1 = nn.Linear(nhid1 * nheads, fusion1_dim)
        self.fusion_att1 = nn.Linear(fusion1_dim * len(adj_list), fusion1_dim)
        self.l1_reg1 = nn.L1Loss(reduction='mean')

        #att 2
        self.attentions2 = nn.ModuleList()
        for i in range(len(adj_list)):
            att_list = nn.ModuleList([GraphAttentionLayer(fusion1_dim, nhid2, dropout=dropout, alpha=alpha, concat=True) for _ in range(nheads)])
            self.attentions2.append(att_list)
            for k, attention in enumerate(self.attentions2):
                self.add_module('adj{}, attention_layer2_{}'.format(i, k), attention)

        #fusion2
        self.integration_att2 = nn.Linear(nhid2 * nheads, nclass)
        self.fusion_att2 = nn.Linear(nclass * len(adj_list), nclass)
        self.l1_reg2 = nn.L1Loss(reduction='mean')


    def forward(self, x, adj_list):
        x = F.dropout(x, self.dropout, training=self.training)

        # Compute output for each adjacency matrix using GAT layers
        output_list = []
        for i, adj in enumerate(adj_list):
            x_i = x
            x_i = torch.cat([att(x, adj) for att in self.attentions1[i]], dim=1)
            x_i = F.dropout(x_i, self.dropout, training=self.training)
            x_i = F.elu(self.integration_att1(x_i))
            output_list.append(x_i)
        output = torch.cat(output_list, dim=1)

        # Apply linear layer for integration with L1 regularization
        output = F.dropout(output, self.dropout, training=self.training)
        output = self.fusion_att1(output.view(output.size(0), -1))
        l1_loss1 = self.l1_reg1(self.fusion_att1.weight, torch.zeros_like(self.fusion_att1.weight))


        output_list2 = []
        for i, adj in enumerate(adj_list):
            x_i = torch.cat([att(output, adj) for att in self.attentions2[i]], dim=1)
            x_i = F.dropout(x_i, self.dropout, training=self.training)
            x_i = F.elu(self.integration_att2(x_i))
            output_list2.append(x_i)
        output2 = torch.cat(output_list2, dim=1)

        # Apply linear layer for integration with L1 regularization
        output2 = F.dropout(output2, self.dropout, training=self.training)
        output2 = self.fusion_att2(output2.view(output.size(0), -1))
        l1_loss2 = self.l1_reg2(self.fusion_att2.weight, torch.zeros_like(self.fusion_att2.weight))

        return F.log_softmax(output2, dim=1), l1_loss1 + l1_loss2

class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

import math

class GraphAttentionLayer(nn.Module):
    """
    Simple GAT layer, similar to https://arxiv.org/abs/1710.10903
    """
    def __init__(self, in_features, out_features, dropout, alpha, concat=True):
        super(GraphAttentionLayer, self).__init__()
        self.dropout = dropout
        self.in_features = in_features
        self.out_features = out_features
        self.alpha = alpha
        self.concat = concat

        self.W = nn.Parameter(torch.empty(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)
        self.a = nn.Parameter(torch.empty(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, h, adj):
        Wh = torch.mm(h, self.W) # h.shape: (N, in_features), Wh.shape: (N, out_features)
        e = self._prepare_attentional_mechanism_input(Wh)

        zero_vec = -9e15*torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)
        h_prime = torch.matmul(attention, Wh)

        if self.concat:
            return F.elu(h_prime)
        else:
            return h_prime

    def _prepare_attentional_mechanism_input(self, Wh):
        # Wh.shape (N, out_feature)
        # self.a.shape (2 * out_feature, 1)
        # Wh1&2.shape (N, 1)
        # e.shape (N, N)
        Wh1 = torch.matmul(Wh, self.a[:self.out_features, :])
        Wh2 = torch.matmul(Wh, self.a[self.out_features:, :])
        # broadcast add
        e = Wh1 + Wh2.T
        return self.leakyrelu(e)

    def __repr__(self):
        return self.__class__.__name__ + ' (' + str(self.in_features) + ' -> ' + str(self.out_features) + ')'


class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [ ]:
import os
import random
import numpy as np
import torch
from torch import Tensor
from torch_geometric.explain.algorithm.utils import clear_masks, set_masks
from torch_geometric.explain.config import ExplanationType, ModelMode, ModelTaskLevel
from typing import Optional, Tuple, Union

from torch_geometric.explain import Explainer, GNNExplainer, PGExplainer

from torch_geometric.explain.config import ModelMode, ModelReturnType
from torch_geometric.utils import get_embeddings

seed = 72
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# Parameter settings
epochs = 500
lr = 0.0005
weight_decay = 5e-5
dropout = 0.3
hidden_dims1 = 16
hidden_dims2 = 16
fusion1_dim = 4
nb_head = 8
alpha = 0.2  # Alpha for the leaky_relu
lambda_l1 =  0.0001
patience = 50

# Load data
sv_path = ""
folder = "# Insert data folder pathway here"
att_name = "# Insert main interaction attribute dataset name here"
edge_list_name = ["# Insert all network edge list dataset names here"]

data = load_multi_data(folder, att_name, edge_list_name)
idx_train, idx_test = data.train_mask, data.test_mask

model = FusionGAT3(
    nfeat=data.features.shape[1],
    nhid1=hidden_dims1,
    nhid2=hidden_dims2,
    fusion1_dim=fusion1_dim,
    nclass=int(data.labels.max()) + 1,
    dropout=dropout,
    alpha=alpha,
    adj_list=data.adj_ls,
    nheads=nb_head
)


model_name = "428.pkl"

# Restore best model
print('Loading model')
model.load_state_dict(torch.load(sv_path+ model_name))



from torch_geometric.explain import Explainer, GNNExplainer

class ModifiedExplainer(Explainer):
    def get_target(self, prediction):
        # Modify the get_target method here
        # You can access the prediction variable directly

        # Perform necessary changes or customizations
        if self.model_config.mode == ModelMode.binary_classification:
            # TODO: Allow customization of the thresholds used below.
            if self.model_config.return_type == ModelReturnType.raw:
                return (prediction > 0).long().view(-1)
            if self.model_config.return_type == ModelReturnType.probs:
                return (prediction > 0.5).long().view(-1)
            assert False

        if self.model_config.mode == ModelMode.multiclass_classification:
            return prediction[0].argmax(dim=-1)

        return prediction


class ModifiedGNNExplainer(GNNExplainer):
    def _train(
        self,
        model: torch.nn.Module,
        x: Tensor,
        edge_index: Tensor,
        *,
        target: Tensor,
        index: Optional[Union[int, Tensor]] = None,
        **kwargs,
    ):
        self._initialize_masks(x, edge_index)

        parameters = []
        if self.node_mask is not None:
            parameters.append(self.node_mask)
        if self.edge_mask is not None:
            set_masks(model, self.edge_mask, edge_index, apply_sigmoid=True)
            parameters.append(self.edge_mask)

        optimizer = torch.optim.Adam(parameters, lr=self.lr)

        for i in range(self.epochs):
            optimizer.zero_grad()

            h = x if self.node_mask is None else x * self.node_mask.sigmoid()
            y_hat, y = model(h, edge_index, **kwargs)[0], target

            if index is not None:
                y_hat, y = y_hat[index], y[index]

            loss = self._loss(y_hat, y)

            loss.backward()
            optimizer.step()

            # In the first iteration, we collect the nodes and edges that are
            # involved into making the prediction. These are all the nodes and
            # edges with gradient != 0 (without regularization applied).
            if i == 0 and self.node_mask is not None:
                if self.node_mask.grad is None:
                    raise ValueError("Could not compute gradients for node "
                                     "features. Please make sure that node "
                                     "features are used inside the model or "
                                     "disable it via `node_mask_type=None`.")
                self.hard_node_mask = self.node_mask.grad != 0.0
            if i == 0 and self.edge_mask is not None:
                if self.edge_mask.grad is None:
                    raise ValueError("Could not compute gradients for edges. "
                                     "Please make sure that edges are used "
                                     "via message passing inside the model or "
                                     "disable it via `edge_mask_type=None`.")
                self.hard_edge_mask = self.edge_mask.grad != 0.0


# class ModifiedPGExplainer(GNNExplainer):
#     def train(
#         self,
#         epoch: int,
#         model: torch.nn.Module,
#         x: Tensor,
#         edge_index: Tensor,
#         *,
#         target: Tensor,
#         index: Optional[Union[int, Tensor]] = None,
#         **kwargs,
#     ):

#         if isinstance(x, dict) or isinstance(edge_index, dict):
#             raise ValueError(f"Heterogeneous graphs not yet supported in "
#                              f"'{self.__class__.__name__}'")

#         if self.model_config.task_level == ModelTaskLevel.node:
#             if index is None:
#                 raise ValueError(f"The 'index' argument needs to be provided "
#                                  f"in '{self.__class__.__name__}' for "
#                                  f"node-level explanations")
#             if isinstance(index, Tensor) and index.numel() > 1:
#                 raise ValueError(f"Only scalars are supported for the 'index' "
#                                  f"argument in '{self.__class__.__name__}'")

#         z = get_embeddings(model, x, edge_index, **kwargs)[-1]

#         self.optimizer.zero_grad()
#         temperature = self._get_temperature(epoch)

#         inputs = self._get_inputs(z, edge_index, index)
#         logits = self.mlp(inputs).view(-1)
#         edge_mask = self._concrete_sample(logits, temperature)
#         set_masks(model, edge_mask, edge_index, apply_sigmoid=True)

#         if self.model_config.task_level == ModelTaskLevel.node:
#             _, hard_edge_mask = self._get_hard_masks(model, index, edge_index,
#                                                      num_nodes=x.size(0))
#             edge_mask = edge_mask[hard_edge_mask]

#         y_hat, y = model(x, edge_index, **kwargs)[0], target

#         if index is not None:
#             y_hat, y = y_hat[index], y[index]

#         loss = self._loss(y_hat, y, edge_mask)
#         loss.backward()
#         self.optimizer.step()

#         clear_masks(model)
#         self._curr_epoch = epoch

#         return float(loss)


##add explainer
explainer = ModifiedExplainer(
    model=model,
    algorithm=ModifiedGNNExplainer(epochs=200),
    explanation_type="model",
    node_mask_type='attributes',
    model_config=dict(
        mode='multiclass_classification',
        task_level='node',
        return_type='log_probs',
    ),
)


loading data
Loading model


<ipython-input-15-1e0519ce22f9>:60: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(sv_path+ model_name))


In [ ]:

#target_label = data.labels
explanation = explainer(data.features, data.adj_ls)#, target = target_label )
print(f'Generated explanations in {explanation.available_explanations}')

Generated explanations in ['node_mask']


In [ ]:
node_mask = explanation.node_mask
np.save(sv_path+f"node_mask_all.npy", node_mask)

In [ ]:
path = f'feature_importance_5top428.png'
feat_label = ["# Insert feature variables here"]
explanation.visualize_feature_importance(sv_path+path, feat_labels = feat_label, top_k=5)
print(f"Feature importance plot has been saved to '{path}'")

Feature importance plot has been saved to 'feature_importance_5top428.png'
